# Kinetic temperature

The kinetic temperature is obtained with the 12CO amplitude :
$$T_{kin}=T_{x}=\frac{5.532}{\ln\left(1+\left(\frac{T_A}{5.532}+0.151\right)^{-1}\right)}$$

In [ ]:
from src.hdu.maps.map import Map
from src.hdu.cubes.cube_co import CubeCO
from src.hdu.tesseract import Tesseract
from src.hdu.maps.grouped_maps import GroupedMaps
from src.hdu.maps.convenient_funcs import get_kinetic_temperature
from src.coordinates.ds9_coords import DS9Coords

In [ ]:
def compute_kinetic_temperature(prefix: str):
    GroupedMaps([(
        "kinetic_temperature", [
            get_kinetic_temperature(amp) for amp in Tesseract.load(
                f"data/Loop4/{prefix}/12co/object_filtered.fits"
            ).to_grouped_maps().amplitude
        ]
    )])#.save(f"data/Loop4/{prefix}/12co/kinetic_temperature.fits")

In [ ]:
compute_kinetic_temperature("N1")

In [ ]:
compute_kinetic_temperature("N2")

In [ ]:
compute_kinetic_temperature("N4")

In [ ]:
compute_kinetic_temperature("p")

# Column density

The column density is obtained using the following equation (Interstellar And Intergalactic Medium, Barbara Ryden and Richard W. Pogge):
\begin{align*}
    N_0\left(^{13}\text{CO}\right)=\int_{-\infty}^\infty T_A\left(^{13}\text{CO}\right)dv\cdot0.8\cdot\frac{g_0}{g_1A_{10}}\cdot\frac{\pi k\nu^2}{hc^3}\left[\left(\frac1{\exp\left(\frac{h\nu}{kT_x}\right)-1}-\frac1{\exp\left(\frac{h\nu}{kT_{rad}}\right)-1}\right)\left(1-\exp\left(-\frac{h\nu}{kT_x}\right)\right)\right]^{-1}
\end{align*}
knowing that
\begin{align*}
    \int_{-\infty}^\infty T_A\left(^{13}\text{CO}\right)dv&=2T_A\sigma\sqrt{\frac\pi2}\text{erf}\left(\frac{\infty}{\sqrt2\sigma}\right)\\
    &=2T_A\sigma\sqrt{\frac\pi2}\\
\end{align*}
as
$$\lim_{x\rightarrow\infty}\text{erf}(x)=1$$
we obtain
$$N_0\left(^{13}\text{CO}\right)=2T_A\sigma\sqrt{\frac\pi2}\cdot0.8\cdot\frac{g_0}{g_1A_{10}}\cdot\frac{\pi k\nu^2}{hc^3}\left[\left(\frac1{\exp\left(\frac{h\nu}{kT_x}\right)-1}-\frac1{\exp\left(\frac{h\nu}{kT_{rad}}\right)-1}\right)\left(1-\exp\left(-\frac{h\nu}{kT_x}\right)\right)\right]^{-1}$$

In [ ]:
import numpy as np
import src.graphinglib as gl
import importlib
import scipy
from typing import Literal
from astropy.constants import M_sun

import src.hdu.maps.convenient_funcs
importlib.reload(src.hdu.maps.convenient_funcs)

In [ ]:
def compute_single_component_column_density_and_13co_opacity(prefix: Literal["N1", "N2", "N4", "p"]):
    cube_12co = CubeCO.load(f"data/Loop4/{prefix}/12co/Loop4{prefix}_wcs.fits")
    cube_13co = CubeCO.load(f"data/Loop4/{prefix}/13co/Loop4{prefix}_13co.fits")
    maps_12co = Tesseract.load(f"data/Loop4/{prefix}/12co/object_filtered.fits").to_grouped_maps()
    maps_13co = Tesseract.load(f"data/Loop4/{prefix}/13co/tesseract.fits").to_grouped_maps()

    # The right gaussians first need to be selected
    # This solution is for single component 13co maps
    assert len(maps_13co.mean) == 1
    mean_12co = np.stack([m.get_reprojection_on(maps_13co.mean[0].header).data for m in maps_12co.mean], axis=0)
    offset_12 = sum([int(line.split(" ")[5][:-1]) if line[12:33] == "was sliced at channel" else 0
                     for line in maps_12co.mean[0].header["COMMENT"]])
    offset_13 = sum([int(line.split(" ")[5][:-1]) if line[13:34] == "was sliced at channel" else 0
                     for line in maps_13co.mean[0].header["COMMENT"]])

    speed_convert_12 = np.vectorize(cube_12co.header.get_value)
    speed_convert_13 = np.vectorize(cube_13co.header.get_value)
    # Compute the diff between the centroid of every gaussian
    diff_array = np.abs(speed_convert_12(mean_12co + offset_12)
                      - speed_convert_13(maps_13co.mean[0].data + offset_13))
    nan_mask = np.isnan(diff_array)     # Apply a nan mask to allow proper argmin use
    diff_array[nan_mask] = 2**15-1      # Remove nans
    min_mask = np.argmin(diff_array, axis=0)
    filter_gaussians = lambda arr: np.take_along_axis(arr, min_mask[np.newaxis, ...], axis=0).squeeze()

    amp_12co_val = np.stack(
        [m.get_reprojection_on(maps_13co.mean[0].header).data for m in maps_12co.amplitude], axis=0
    )
    amp_12co_unc = np.stack(
        [m.get_reprojection_on(maps_13co.mean[0].header).uncertainties for m in maps_12co.amplitude], axis=0
    )

    amplitude_correction_factor_13co = 0.43
    src.hdu.maps.convenient_funcs.get_13co_column_density(
        stddev_13co=maps_13co.stddev[0]*np.abs(cube_13co.header["CDELT3"]/1000),
        antenna_temperature_13co=maps_13co.amplitude[0]/amplitude_correction_factor_13co,
        antenna_temperature_12co=Map(filter_gaussians(amp_12co_val), filter_gaussians(amp_12co_unc))
    ).save(f"data/Loop4/{prefix}/13co/{prefix}_column_density.fits")
    src.hdu.maps.convenient_funcs.get_13co_opacity(
        stddev_13co=maps_13co.stddev[0]*np.abs(cube_13co.header["CDELT3"]/1000),
        antenna_temperature_13co=maps_13co.amplitude[0]/amplitude_correction_factor_13co,
        antenna_temperature_12co=Map(filter_gaussians(amp_12co_val), filter_gaussians(amp_12co_unc))
    ).save(f"data/Loop4/{prefix}/13co/{prefix}_opacity.fits")

In [ ]:
compute_single_component_column_density_and_13co_opacity("N1")

In [ ]:
compute_single_component_column_density_and_13co_opacity("N2")

In [ ]:
compute_single_component_column_density_and_13co_opacity("N4")

In [ ]:
# Loop4p multiple components
import scipy.optimize

cube_12co = CubeCO.load("data/Loop4/p/12co/Loop4p_wcs.fits")
cube_13co = CubeCO.load("data/Loop4/p/13co/Loop4p_13co.fits")
maps_12co = Tesseract.load("data/Loop4/p/12co/object_filtered.fits").to_grouped_maps()
maps_13co = Tesseract.load("data/Loop4/p/13co/object_filtered.fits").to_grouped_maps()

mean_12co = np.stack([m.get_reprojection_on(maps_13co.mean[0].header).data for m in maps_12co.mean], axis=0)
mean_13co = np.stack([m.data for m in maps_13co.mean], axis=0)
ampl_12co_val = np.stack([m.get_reprojection_on(maps_13co.amplitude[0].header).data
                          for m in maps_12co.amplitude], axis=0)
ampl_12co_unc = np.stack([m.get_reprojection_on(maps_13co.amplitude[0].header).uncertainties
                          for m in maps_12co.amplitude], axis=0)

ordered_stddev_13co = np.full([*mean_13co.shape, 2], np.NAN)
ordered_amplitude_13co = np.full([*mean_13co.shape, 2], np.NAN)
ordered_amplitude_12co = np.full([*mean_13co.shape, 2], np.NAN)

speed_convert_12 = np.vectorize(cube_12co.header.get_value)
speed_convert_13 = np.vectorize(cube_13co.header.get_value)

def minimize(target: np.ndarray, ref: np.ndarray):
    """
    Minimizes the distance between two groups of points and gives the matching indices.
    """
    # Create a cost matrix where the element at position (i, j) represents the difference between list1[i] and list2[j]
    cost_matrix = np.abs(np.subtract.outer(target[~np.isnan(target)], ref[~np.isnan(ref)]))
    # Use linear_sum_assignment to find the optimal assignment
    row_indices, col_indices = scipy.optimize.linear_sum_assignment(cost_matrix)
    # Create a list of tuples representing the pairs
    pairs = list(zip(row_indices, col_indices))
    # Check if the pairs are close enough, otherwise the pair is considered invalid
    velocity_upper_limit = 100
    valid_pairs = []
    for pair in pairs:
        if np.abs(target[pair[0]] - ref[pair[1]]) < velocity_upper_limit:
            valid_pairs.append(pair)
    return valid_pairs

for y in range(mean_13co.shape[1]):
    for x in range(mean_13co.shape[2]):
        if not np.isnan(mean_13co[0,y,x]):
            matches = minimize(speed_convert_13(mean_13co[:,y,x]+400), speed_convert_12(mean_12co[:,y,x]+500))
            for match in matches:
                ordered_stddev_13co[match[0],y,x] = [
                    maps_13co.stddev[match[0]].data[y,x],
                    maps_13co.stddev[match[0]].uncertainties[y,x]
                ]
                ordered_amplitude_13co[match[0],y,x] = [
                    maps_13co.amplitude[match[0]].data[y,x],
                    maps_13co.amplitude[match[0]].uncertainties[y,x]
                ]
                ordered_amplitude_12co[match[0],y,x] = [
                    ampl_12co_val[match[1],y,x],
                    ampl_12co_unc[match[1],y,x]
                ]

amplitude_correction_factor_13co = 0.43
column_densities = []
opacities = []
for i in range(mean_13co.shape[0]):
    stddev_13co = Map(
        data=ordered_stddev_13co[i,:,:,0],
        uncertainties=ordered_stddev_13co[i,:,:,1],
        header=maps_13co.mean[0].header,
    ) * np.abs(cube_13co.header["CDELT3"]/1000)
    antenna_temperature_13co = Map(
        data=ordered_amplitude_13co[i,:,:,0],
        uncertainties=ordered_amplitude_13co[i,:,:,1],
        header=maps_13co.mean[0].header,
    ) / amplitude_correction_factor_13co
    antenna_temperature_12co = Map(
        data=ordered_amplitude_12co[i,:,:,0],
        uncertainties=ordered_amplitude_12co[i,:,:,1],
        header=maps_13co.mean[0].header,
    )

    column_densities.append(
        src.hdu.maps.convenient_funcs.get_13co_column_density(
            stddev_13co=stddev_13co,
            antenna_temperature_13co=antenna_temperature_13co,
            antenna_temperature_12co=antenna_temperature_12co
        )
    )
    opacities.append(
        src.hdu.maps.convenient_funcs.get_13co_opacity(
            stddev_13co=stddev_13co,
            antenna_temperature_13co=antenna_temperature_13co,
            antenna_temperature_12co=antenna_temperature_12co,
        )
    )

# GroupedMaps([("column_density", column_densities)]).save("data/Loop4/p/13co/p_column_density.fits")
# GroupedMaps([("opacity", opacities)]).save("data/Loop4/p/13co/p_opacity.fits")

## H2 column density

In [ ]:
def calculate_h2_column_density(
    prefix: Literal["N1", "N2", "N4", "p"],
):
    tess = Tesseract.load(f"data/Loop4/{prefix}/12co/object_filtered.fits")
    cube = CubeCO.load(f"data/Loop4/{prefix}/12co/Loop4{prefix}_wcs.fits")
    X_CO = 2.5e20
    gm = tess.to_grouped_maps()
    n_h2 = []
    for amplitude, stddev in zip(gm.amplitude, gm.stddev):
        n_h2.append(
            src.hdu.maps.convenient_funcs.integrate_gaussian(
                amplitude_map=amplitude,
                stddev_map=stddev * np.abs(cube.header["CDELT3"]) / 1000
            ) * X_CO
        )
    GroupedMaps([("H2_column_density", n_h2)])#.save(f"data/Loop4/{prefix}/12co/{prefix}_H2_column_density.fits")
    Map(
        data=np.nansum([m.data for m in n_h2], axis=0),
        uncertainties=np.nansum([m.uncertainties for m in n_h2], axis=0),
        header=n_h2[0].header,
    ).num_to_nan()#.save(f"data/Loop4/{prefix}/12co/{prefix}_H2_column_density_total.fits")

def calculate_h2_column_density_with_13co(
        prefix: str,
):
    if prefix == "p":
        column_densities = GroupedMaps.load("data/Loop4/p/13co/p_column_density.fits").column_density
        (2.2e6 * Map(
            data=np.nansum([m.data for m in column_densities], axis=0),
            uncertainties=np.nansum([m.uncertainties for m in column_densities], axis=0),
            header=column_densities[0].header,
        ).num_to_nan())#.save(f"data/Loop4/{prefix}/13co/{prefix}_H2_column_density_total_13co.fits")

    else:
        column_density = Map.load(f"data/Loop4/{prefix}/13co/{prefix}_column_density.fits")
        (2.2e6 * column_density)#.save(f"data/Loop4/{prefix}/13co/{prefix}_H2_column_density_total_13co.fits")

In [ ]:
calculate_h2_column_density("N1")
calculate_h2_column_density("N2")
calculate_h2_column_density("N4")
calculate_h2_column_density("p")

In [ ]:
calculate_h2_column_density_with_13co("N1")
calculate_h2_column_density_with_13co("N2")
calculate_h2_column_density_with_13co("N4")
calculate_h2_column_density_with_13co("p")

## Cloud mass

In [ ]:
def calculate_cloud_mass(
    prefix: Literal["N1", "N2", "N4", "p"],
):
    """
    Gives the cloud's mass in kg.
    """
    alpha = 30 / 3600 * (2*np.pi)/360
    if prefix in ["N1", "p"]:       # These two clouds were binned 2x2 whuch results in
        alpha *= 2
    D = 370 * scipy.constants.parsec * 100
    mu = 2.4
    m_H = scipy.constants.proton_mass + scipy.constants.electron_mass

    n_h2 = Map.load(f"data/Loop4/{prefix}/12co/{prefix}_H2_column_density_total.fits")
    sum_n_h2 = np.array([
        np.nansum(n_h2.data),
        np.nansum(n_h2.uncertainties),
    ])
    M = (alpha * D)**2 * mu * m_H * sum_n_h2
    return M

In [ ]:
fm = lambda x: f"({x[0]:.5e} ± {x[1]:.5e})"
fm_solar = lambda x: f"({x[0]/M_sun.value:.5e} ± {x[1]/M_sun.value:.5e})"
for cloud in ["N1", "N2", "N4", "p"]:
    m = calculate_cloud_mass(cloud)
    print(f"Cloud {cloud:2}: M(H2)={fm(m):27} kg, M(CO)={fm(3e-6 * m):27} kg")
    print(f"                {fm_solar(m):27} Ms, M(CO)={fm_solar(3e-6 * m):27} Ms")
    print()

### Table

In [ ]:
from uncertainties import ufloat

cloud_masses = [[str(ufloat(*calculate_cloud_mass(cloud)) / 1e31), str(ufloat(*calculate_cloud_mass(cloud)) * 3e-6 / 1e26)]
                for cloud in ["N1", "N2", "N4", "p"]]
cloud_masses = [[s.replace("+/-", " ± ") for s in row] for row in cloud_masses]

table_fig = gl.SmartFigure(
    remove_axes=True,
    size=(3.5, 2),
    elements=[
        gl.Table(
            cell_text=cloud_masses,
            col_labels=["M(H$_2$) [$10^{31}$ kg]", "M(CO) [$10^{26}$ kg]"],
            row_labels=["N1", "N2", "N4", "p"],
            cell_align="center",
        )
    ]
)
table_fig.show()
# table_fig.save("figures/Loop4/second_results/cloud_masses.pdf")

# Presentation figures

In [ ]:
import graphinglib as gl
import matplotlib.pyplot as plt
from astropy.wcs import WCS

In [ ]:
cube = CubeCO.load("data/Loop4/N1/12co/Loop4N1_wcs.fits")[575:750,:,:]
fig = plt.figure(figsize=(6,6))
fig.subplots_adjust(left=0.14, bottom=0.06, right=0.9, top=0.98, wspace=None, hspace=None)  # left, right, top, bottom
header = cube.header
ax = fig.add_subplot(111, projection=WCS(header.flatten(0)))
anim = cube.data.plot_mpl(fig, ax,
    cbar_limits=(0,10),
    time_interval=30,
    xlabel="Ascension droite",
    ylabel="Déclinaison",
    cbar_label="Intensité [u. arb.]"
)
plt.show()
# anim.save("figures/Loop4/Loop4N1_wcs.gif", writer="imagemagick", dpi=150)

In [ ]:
m_speed = GroupedMaps.load("speed_maps/N1_speed.fits").centroid_speed

fig = plt.figure(figsize=(6,6))
fig.subplots_adjust(left=0.14, bottom=0.0, right=0.88, top=1.04, wspace=None, hspace=None)  # left, right, top, bottom
header = m_speed[0].header
ax = fig.add_subplot(111, projection=WCS(header.flatten(0)))
cbar = plt.colorbar(ax.imshow(m_speed[0].data), fraction=0.057, pad=0.03)
ax.tick_params(axis='both', direction='in')

plt.xlabel("Ascension droite")
plt.ylabel("Déclinaison")
cbar.set_label("Vitesse des centroïdes [km s$^{-1}$]")
# %matplotlib inline
# plt.show()
plt.savefig("figures/Loop4/N1_speed.png", dpi=600)

In [ ]:
column_densities = GroupedMaps.load("data/Loop4/p/12co/p_H2_column_density.fits").H2_column_density
fig = plt.figure(figsize=(12,5.5))
fig.subplots_adjust(left=0.07, bottom=0.05, right=0.9, top=1, wspace=0.3, hspace=None)  # left, right, top, bottom
header = column_densities[0].header
vmin, vmax = 1e19, 3e21
axs = []

for i, map_ in enumerate(column_densities[:-1]):
    axs.append(fig.add_subplot(1, 3, i+1, projection=WCS(header)))
    imshow = axs[-1].imshow(map_.data, vmin=vmin, vmax=vmax)
    axs[-1].tick_params(axis='both', direction='in')
    plt.xlabel(" ")
    plt.ylabel(" ")

cbar_ax = fig.add_axes([0.92, 0.155, 0.022, 0.75])
cbar = fig.colorbar(imshow, cax=cbar_ax)

fig.supxlabel("Ascension droite", size=12)
fig.supylabel("Déclinaison")
cbar.set_label("Densité de colonne du HI [cm $^{-2}$]")
# %matplotlib inline
# plt.show()
plt.savefig("figures/Loop4/column_density_HI_13co.png", dpi=600)

In [ ]:
column_densities = GroupedMaps.load("data/Loop4/p/13co/p_column_density.fits").column_density
fig = plt.figure(figsize=(8,5.5))
fig.subplots_adjust(left=0.1, bottom=0.05, right=0.85, top=1, wspace=0.2, hspace=None)  # left, right, top, bottom
header = column_densities[0].header
vmin, vmax = 0, 4e14
axs = []

for i, map_ in enumerate(column_densities[:-1]):
    axs.append(fig.add_subplot(1, 2, i+1, projection=WCS(header)))
    imshow = axs[-1].imshow(map_.data, vmin=vmin, vmax=vmax)
    axs[-1].tick_params(axis='both', direction='in')
    plt.xlabel(" ")
    plt.ylabel(" ")

cbar_ax = fig.add_axes([0.90, 0.155, 0.022, 0.75])
cbar = fig.colorbar(imshow, cax=cbar_ax)

fig.supxlabel("Ascension droite", size=12)
fig.supylabel("Déclinaison")
cbar.set_label("Densité de colonne du $^{13}$CO [cm $^{-2}$]")
# %matplotlib inline
# plt.show()
plt.savefig("figures/Loop4/column_density_13co.png", dpi=600)

# Figures

In [ ]:
import numpy as np
import src.graphinglib as gl
import pyregion
import warnings

from src.hdu.maps.grouped_maps import GroupedMaps
from src.hdu.maps.map import Map
from src.hdu.cubes.cube_co import Cube

## Kinetic temperature

In [ ]:
figs = []
for cloud, (i, j) in zip(["N1", "N2", "N4", "p"], [(1, 2), (1, 2), (1, 3), (2, 2)]):
    cloud_heatmaps = []
    maps = GroupedMaps.load(f"data/Loop4/{cloud}/12co/kinetic_temperature.fits").kinetic_temperature
    for map_ in maps:
        hm = map_.data.plot
        hm.set_color_bar_params(label="Kinetic Temperature [K]")
        cloud_heatmaps.append(hm)
    fig = gl.SmartFigureWCS(maps[0].header.wcs_object, i, j, x_label="Right Ascension", y_label="Declination",
                            elements=cloud_heatmaps, size=(5*j, 4*i), aspect_ratio="auto")
    fig.set_ticks(number_of_x_ticks=4, minor_x_tick_frequency=5, minor_y_tick_frequency=5)
    figs.append(fig)
    fig.save(f"figures/Loop4/first_results/{cloud}/kinetic_temperature.png", dpi=600)

for fig in figs:
    fig.show()


## Speed map histograms

In [ ]:
figs = []
for cloud in ["N1", "N2", "N4", "p"]:
    maps = GroupedMaps.load(f"data/Loop4/speed_maps/{cloud}_speed.fits").centroid_speed
    hist = gl.Histogram(
        data=np.concatenate([m.data[~np.isnan(m.data)] for m in maps]),
        # data=np.concatenate([m.data.flatten() for m in maps]),
        number_of_bins=20,
        normalize=False,
    )
    fig = gl.SmartFigure(elements=[hist], title=cloud)
    fig.set_ticks(x_tick_spacing=0.5)
    figs.append(fig)
fig = gl.SmartFigure(
    2,
    2,
    elements=figs,
    size=(10, 8),
    x_label="Centroid Speed [km s$^{-1}$]",
    y_label="Number of Pixels [-]"
)
fig[1,1].x_lim = (-2.2, 1.99)
# fig.show()
# fig.save("figures/Loop4/first_results/speed_histograms.png", dpi=600)


## Linewidths

In [ ]:
figs = []
for cloud, (i, j) in zip(["N1", "N2", "N4", "p"], [(1, 2), (1, 2), (1, 3), (2, 2)]):
    cloud_heatmaps = []
    cube_12co = CubeCO.load(f"data/Loop4/{cloud}/12co/Loop4{cloud}_wcs.fits")
    maps_12co = Tesseract.load(f"data/Loop4/{cloud}/12co/object_filtered.fits").to_grouped_maps()
    linewidths = [map_ * np.abs(cube_12co.header["CDELT3"]/1000) * 2*np.sqrt(2*np.log(2)) for map_ in maps_12co.stddev]
    for map_ in linewidths:
        hm = map_.data.plot
        hm.set_color_bar_params(label="Linewidth (FWHM) [km s$^{-1}$]")
        cloud_heatmaps.append(hm)
    fig = gl.SmartFigureWCS(linewidths[0].header.wcs_object, i, j, x_label="Right Ascension", y_label="Declination",
                            elements=cloud_heatmaps, size=(5*j, 4*i), aspect_ratio="auto")
    fig.set_ticks(number_of_x_ticks=4, minor_x_tick_frequency=5, minor_y_tick_frequency=5)
    figs.append(fig)
    # fig.save(f"figures/Loop4/second_results/{cloud}/linewidths.png", dpi=600)
    # GroupedMaps([("linewidth", linewidths)]).save(f"figures/Loop4/second_results/{cloud}/linewidths.fits")

for fig in figs:
    fig.show()


## Opacity

In [ ]:
figs = []
for cloud in ["N1", "N2", "N4"]:
    map_ = Map.load(f"data/Loop4/{cloud}/13co/{cloud}_opacity.fits")
    hm = map_.data.plot
    hm.set_color_bar_params(label="Opacity [-]")
    fig = gl.SmartFigureWCS(map_.header.wcs_object, 1, 1, x_label="Right Ascension", y_label="Declination",
                            elements=[hm], size=(5, 4), aspect_ratio="auto")
    fig.set_ticks(number_of_x_ticks=3, minor_x_tick_frequency=5, minor_y_tick_frequency=5)
    if cloud == "N1":
        fig[0][0].color_map_range = 0.04, 0.31
    figs.append(fig)
    # fig.save(f"figures/Loop4/second_results/{cloud}/opacity.png", dpi=600)

p_heatmaps = []
p_maps = GroupedMaps.load(f"data/Loop4/p/13co/p_opacity.fits").opacity
for map_ in p_maps:
    hm = map_.data.plot
    hm.set_color_bar_params(label="Opacity [-]")
    p_heatmaps.append(hm)
fig = gl.SmartFigureWCS(p_maps[0].header.wcs_object, 1, 3, x_label="Right Ascension", y_label="Declination",
                        elements=p_heatmaps, size=(12, 4), aspect_ratio="auto")
fig.set_ticks(number_of_x_ticks=4, minor_x_tick_frequency=5, minor_y_tick_frequency=5)
figs.append(fig)
# fig.save(f"figures/Loop4/second_results/p/opacity.png", dpi=600)

for fig in figs:
    fig.show()


## IR excess

### Column density with HI

In [ ]:
def calculate_column_density(arr: np.ndarray) -> np.ndarray:
    """
    Calculates the column density for each pixel in the given array. Each gaussian's area is computed using the error
    function, and the total column density is obtained by summing the areas of all gaussians in each pixel.

    Parameters
    ----------
    arr : np.ndarray
        A 4D array with shape (n_y, n_x, n_gaussians, 3), where the last dimension contains the amplitude, mean and
        sigma of the gaussian.

    Returns
    -------
    np.ndarray
        A 2D array with shape (n_y, n_x) containing the column density for each pixel, obtained by summing the gaussian
        areas for each gaussian in the pixel.

    See Also
    --------
    applications/co/graph_gaussians/rohsa_gaussian.ipynb for more details on the formula used.
    """
    column_densities = 1.82e18 * 2 * arr[:,:,:,0] * arr[:,:,:,2] * np.sqrt(np.pi/2)
    return column_densities.sum(axis=2)

def reshape_array(arr: np.ndarray) -> np.ndarray:
    """
    Reshapes the input 2D array to a 4D array with shape (n_y, n_x, n_gaussians, 3), where the last dimension contains
    the amplitude, mean and sigma of the gaussian. The n_y and n_x sizes are inferred from the last element of the input
    array.

    Parameters
    ----------
    arr : np.ndarray
        A 2D array corresponding to the output of the ROHSA program. Along the second axis, the first two elements are
        y and x coordinates, and the rest are the parameters of the gaussian (amplitude, mean, sigma).

    Returns
    -------
    np.ndarray
        A 4D array with shape (n_y, n_x, n_gaussians, 3), where the last dimension contains the amplitude, mean and
        sigma of the gaussian.
    """
    n_y, n_x = arr[-1,:2].astype(int) + 1
    reshaped_arr = arr.reshape(arr.shape[0]//7, 7, -1)      # reshape to (n_pixels, 7, n_params)
    reshaped_arr = reshaped_arr.reshape(n_y, n_x, 7, -1)    # (n_y, n_x, 7, n_params)
    reshaped_arr = reshaped_arr[:, :, :, 2:]  # remove the first two parameters (y and x coordinates)
    return reshaped_arr

In [ ]:
warnings.filterwarnings("ignore", module="astropy.wcs.wcs")

# cube = Cube.load("data/HI/spider_hi.fits").bin((1, 8, 8))
# cube.save("data/HI/spider_hi_binned.fits")
cube = Cube.load("data/HI/spider_hi_binned.fits")
map_header = cube.header.flatten(0)
# gl.SmartFigureWCS(projection=map_header.wcs_object, elements=[cube.data.plot]).show()

background_data = np.loadtxt("data/HI/rohsa_fits/DF_gauss_run_0.dat")
background_column_density = Map(calculate_column_density(reshape_array(background_data)), header=map_header)
regions = pyregion.open("data/HI/spiderZones_exact.reg")
regions_dict = {
    "sud": (slice(113, 175), slice(30, 170)),
    "nord": (slice(32, 88), slice(49, 163)),
    "coeur": (slice(88, 113), slice(88, 113)),
    "est": (slice(88, 113), slice(15, 88)),
    "ouest": (slice(88, 113), slice(113, 201)),
}
for region_name, key in regions_dict.items():
    region_data = np.loadtxt(f"data/HI/rohsa_fits/DF_gauss_run_{region_name}.dat")
    region_column_density = calculate_column_density(reshape_array(region_data))
    background_column_density.data[key] = region_column_density

patches = regions.as_imagecoord(header=map_header).get_mpl_patches_texts()[0]
polygons = [gl.Polygon(patch.get_xy()[:-1], fill=False, edge_color=patch.get_edgecolor()) for patch in patches]
column_density_fig = gl.SmartFigureWCS(
    projection=map_header.wcs_object,
    x_label="Right Ascension",
    y_label="Declination",
    size=(8, 7),
    elements=[
        background_column_density.data.plot,
        *polygons,
    ],
)
column_density_fig[0][0].set_color_bar_params(label="Column density [$10^{20}$cm$^{-2}$]")
column_density_fig[0][0].color_map_range = (0.5e20, 9.5e20)
column_density_fig[0][0].color_map = "inferno"
column_density_fig.set_grid(show_on_top=True, line_width=1, color="white")
column_density_fig.set_tick_params(axis="both", color="white")
column_density_fig.show()
# background_column_density.save("data/HI/spider_hi_background_column_density_merged.fits")


### IRIS mosaic

In [ ]:
iris_maps = [
    Map.load(f"data/HI/iris/{field}B4H0.fits") for field in ["I414", "I425", "I426"]
]
for map_ in iris_maps:
    for keyword in ["CRVAL3", "CRPIX3", "CTYPE3", "CDELT3"]:
        if keyword in map_.header:
            map_.header.remove(keyword)
iris_maps_reprojected = [map_.get_reprojection_on(background_column_density.header) for map_ in iris_maps]
iris_map = iris_maps_reprojected[0].copy()
for map_ in iris_maps_reprojected[1:]:
    iris_map.data[np.isnan(iris_map.data)] = map_.data[np.isnan(iris_map.data)]

fig = gl.SmartFigure(elements=[iris_map.data.plot])
fig[0][0].color_map_range = 0, 10
fig.show()
# iris_map.save("data/HI/iris_ir_excess.fits")

The iras IR excess map is obtained from https://irsa.ipac.caltech.edu/cgi-bin/ISSA/nph-issa?objstr=10%3A20%3A15+eq+73%3A55%3A25+eq&size=12.5+deg&iraspsc=1&coordinate_grid=1&band=4&submit=submit.

The IRIS corrected maps are obtained from https://irsa.ipac.caltech.edu/cgi-bin/Atlas/nph-atlas?mission=IRIS&hdr_location=%2Fwork%2FTMP_WGRSaE_25947%2FAtlas%2F10h_20m_15.00s_%2B73d_55m_25.0s_Equ_J2000_25528.v0001&collection_desc=Improved+Reprocessing+of+the+IRAS+Survey+%28IRIS%29&region.x=200&region.y=218&locstr=10h+20m+15.00s+%2B73d+55m+25.0s+Equ+J2000&searchregion=&radius=6.25&regSize=12.5&covers=on.

### IR excess deviation

In [ ]:
iris_ir_excess = Map.load("data/HI/iris_ir_excess.fits")
iris_ir_excess = iris_ir_excess.get_reprojection_on(map_header)
iris_ir_excess_hm = iris_ir_excess.data.plot
iris_ir_excess_hm.color_map = "grey"
iris_ir_excess_hm.color_map_range = (0, 8)
iris_ir_excess_hm.show_color_bar = False

cropped_background = background_column_density.copy()
cropped_background.data[cropped_background.data < 3e20] = np.nan

superposed_fig = gl.SmartFigureWCS(
    projection=map_header.wcs_object,
    x_label="Right Ascension",
    y_label="Declination",
    size=(8, 7),
    elements=[
        iris_ir_excess_hm,
        gl.Heatmap(
            cropped_background.data / 1e20,
            color_map="inferno",
            origin_position="lower",
            color_map_range=(0.5, 9.5),
            alpha_value=1,
            show_color_bar=True,
        ),
    ],
)
superposed_fig[0][1].set_color_bar_params(label="Column density [$10^{20}$cm$^{-2}$]")
# superposed_fig.set_grid(show_on_top=True, line_width=1, color="white")
# superposed_fig.set_tick_params(axis="both", color="white")
superposed_fig.show()


In [ ]:
scatter_data = np.column_stack((background_column_density.data.flatten() / 1e20, iris_ir_excess.data.flatten()))
scatter_data = scatter_data[np.argsort(scatter_data[:, 0])]
scatter = gl.Scatter(
    x_data=scatter_data[:,0],
    y_data=scatter_data[:,1],
    marker_size=0.7,
    face_color="black",
)

sigma_multiplier = 3
lin_func = lambda x: 0.56 * x + 0.63
fit = gl.Curve.from_function(lin_func, scatter.x_data.min(), scatter.x_data.max(),
                             label="Reach et al. 1998 slope:\n$y=0.56x+0.63$")

scatter_std_sample = scatter.create_slice_x(0, 2)
threshold = np.std(scatter_std_sample.y_data - lin_func(scatter_std_sample.x_data)) * sigma_multiplier
fit.add_error_curves(
    threshold,
    error_curves_line_style="--",
)

background_column_density_cropped = background_column_density.copy() / 1e20
excess_map = iris_ir_excess - lin_func(background_column_density / 1e20)
background_column_density_cropped.data[excess_map.data < threshold] = np.nan

fig = gl.SmartFigure(
    2,
    1,
    elements=[
        gl.SmartFigure(
            x_label="Column density [10$^{20}$cm$^{-2}$]",
            y_label=r"IRIS 100 $\mu$m excess [MJy sr$^{-1}$]",
            elements=[scatter, fit],
            y_lim=(0, 10),
            x_lim=(0, None),
        ),
        gl.SmartFigureWCS(
            projection=map_header.wcs_object,
            elements=[
                iris_ir_excess_hm,
                background_column_density_cropped.data.plot,
            ],
            x_label="Right Ascension",
            y_label="Declination",
            aspect_ratio="equal",
        )
    ],
    size=(8, 10),
    height_ratios=(1, 2),
)
fig[0].set_ticks(minor_x_tick_spacing=0.25, x_tick_spacing=1, minor_y_tick_spacing=0.5)
fig[1][0][1].color_map = "inferno"
fig[1][0][1].set_color_bar_params(label="Column density [$10^{20}$cm$^{-2}$]")
from matplotlib.lines import Line2D
fig[0].set_custom_legend(
    [Line2D([], [], linestyle="--", label=rf"$\pm{sigma_multiplier}\sigma$", color=gl.get_color())]
)
fig.show()
# fig.save("figures/HI/spider_hi_excess.png", dpi=600)
